<a href="https://colab.research.google.com/github/thestranger-coder/starter-python-telegram-bot/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing Packages

!pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.9/576.9 kB 5.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for telethon: filename=Telethon-1.34.0-py3-none-any.whl size=649201 sha256=45892c9853d9b64510e908b33b8e3ff8bbd58956fdea00809bc6c4584b6ce72e
  Stored in directory: /root/.cache/pip/wheels/d5/28/56/edfb090fbbcd57add61a49dfe194c8eff22b5d4828008f2e62
  Created wheel for pyaes: filename=pyaes-1.6.1-py3-none-any.whl size=26347 sha256=d67b6fdbb21d1151fa44098c5fa2973d57f80344e45940f8d905473c8dd0a0c9
  Stored in directory: /root/.cache/pip/wheels/d6/84/5f/ea6aef85a93c7e1922486369874f4740a5642d261e09c59140
Successfully built telethon pyaes


In [ ]:
import io
import json
import requests
import asyncio
import os
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
from telegram import ReplyKeyboardMarkup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from telethon.sync import TelegramClient
from telethon.tl.types import DocumentAttributeFilename

# Path to the service account credentials JSON file
SERVICE_ACCOUNT_CREDENTIALS_FILE = 'service_account.json'
# Path to the file containing your chat ID
CHAT_ID_FILE = 'MyChatID.txt'
# Path to the file containing Google Drive folder ID
GOOGLE_DRIVE_FOLDER_ID_FILE = 'GoogleDriveFolderID.txt'
# Path to the file containing Telegram bot token
TELEGRAM_TOKEN_FILE = 'telegram_token.txt'

# Replace the following values with your own API credentials for Telethon
api_id = '25615662'
api_hash = '644e66e47b471bd3654271cd7092f749'

# Replace these with your own session name and phone number for Telethon
session_name = 'my_telegram_session'
phone_number = '+917417331224'  # Your phone number associated with the Telegram account

# Size limit for files (in bytes)
max_file_size = 2 * 1024 * 1024 * 1024  # 2 GB


# Function to initialize Google Drive service with service account credentials
def initialize_drive_service():
  with open(SERVICE_ACCOUNT_CREDENTIALS_FILE, 'r') as f:
    service_account_info = json.load(f)
  creds = Credentials.from_service_account_info(service_account_info)
  service = build('drive', 'v3', credentials=creds)
  return service


# Function to read a single line from a file
def read_single_line_from_file(file_path):
  with open(file_path, 'r') as file:
    return file.readline().strip()


# Function to start the bot
def start(update, context):
  chat_id = update.message.chat_id
  authorized_chat_id = read_single_line_from_file(CHAT_ID_FILE)
  if str(chat_id) != authorized_chat_id:
    context.bot.send_message(chat_id=chat_id, text="You are not authorized!")
    return
  update.message.reply_text(
      'Welcome to the Direct Download Link Uploader Bot!')


# Function to handle direct download link and upload the file
def upload_link(update, context):
  chat_id = update.message.chat_id
  authorized_chat_id = read_single_line_from_file(CHAT_ID_FILE)
  if str(chat_id) != authorized_chat_id:
    context.bot.send_message(chat_id=chat_id, text="You are not Authorized!")
    return

  choice = update.message.text

  if choice.startswith("http") or choice.startswith("https"):
    context.user_data['direct_download_link'] = choice
    file_name = choice.split('/')[-1]
    context.user_data['direct_download_file_name'] = file_name

    keyboard = [['Google Drive', 'Telegram']]
    reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
    update.message.reply_text('Where do you want to download the file?',
                              reply_markup=reply_markup)
    context.user_data['state'] = 'WAITING_FOR_DOWNLOAD_CHOICE'

  else:
    update.message.reply_text('Please provide a valid direct download link.')


# Function to handle user choice from the menu
def handle_download_choice(update, context):
  choice = update.message.text

  if 'state' in context.user_data and context.user_data[
      'state'] == 'WAITING_FOR_DOWNLOAD_CHOICE':

    if choice not in ['Google Drive', 'Telegram']:
      update.message.reply_text('Please select a valid option.')
      return

    link = context.user_data.get('direct_download_link')

    if choice == 'Google Drive':
      download_to_drive(update, context, link)
    elif choice == 'Telegram':
      download_to_telegram(update, context, link)

    context.user_data.clear()

  else:
    update.message.reply_text('Invalid state. Please start again.')


# Function to download the file to Google Drive
def download_to_drive(update, context, link):
  chat_id = update.message.chat_id
  try:
    folder_id = read_single_line_from_file(GOOGLE_DRIVE_FOLDER_ID_FILE)
    drive_service = initialize_drive_service()
    file_name = context.user_data.get('direct_download_file_name')

    file_metadata = {'name': file_name, 'parents': [folder_id]}

    response = requests.get(link, stream=True)
    response.raise_for_status()
    media_body = MediaIoBaseUpload(io.BytesIO(response.content),
                                   mimetype='application/octet-stream',
                                   chunksize=-1)
    uploaded_file = drive_service.files().create(
        body=file_metadata, media_body=media_body,
        fields='id,name,mimeType').execute()

    file_id = uploaded_file.get('id')
    file_url = f'https://drive.google.com/uc?id={file_id}'

    completion_message = f"Upload to Google Drive completed! You can access the file [here]({file_url})."
    context.bot.send_message(chat_id=chat_id,
                             text=completion_message,
                             parse_mode="Markdown")

  except requests.RequestException as e:
    error_message = f"Failed to get http URL content: {str(e)}"
    context.bot.send_message(chat_id=chat_id, text=error_message)

  except Exception as e:
    error_message = f"An error occurred: {str(e)}"
    context.bot.send_message(chat_id=chat_id, text=error_message)


# Function to download the file to Telegram using Telethon
async def download_to_telegram_telethon(update, context, link):
  chat_id = update.message.chat_id
  try:
    response = requests.get(link)
    file_name = context.user_data.get('direct_download_file_name')

    # Create a Telegram client instance
    client = TelegramClient(session_name, api_id, api_hash)

    # Connect the client
    await client.start(phone_number)

    # Upload the file
    async with client:
      # Check if the file size is within the limit
      if len(response.content) <= max_file_size:
        # Send the file
        await client.send_file(
            chat_id,
            io.BytesIO(response.content),
            attributes=[DocumentAttributeFilename(file_name)])
        context.bot.send_message(chat_id=chat_id,
                                 text="Download to Telegram completed!")
      else:
        context.bot.send_message(chat_id=chat_id,
                                 text="File size exceeds the limit of 2GB.")

  except requests.RequestException as e:
    error_message = f"Failed to get http URL content: {str(e)}"
    context.bot.send_message(chat_id=chat_id, text=error_message)

  except Exception as e:
    error_message = f"An error occurred: {str(e)}"
    context.bot.send_message(chat_id=chat_id, text=error_message)


# Function to download the file to Telegram
def download_to_telegram(update, context, link):
  loop = asyncio.new_event_loop()
  asyncio.set_event_loop(loop)
  loop.run_until_complete(download_to_telegram_telethon(update, context, link))


# Function to handle file uploads to Telegram
def handle_file_upload(update, context):
  file_obj = update.message.document
  file_name = file_obj.file_name
  file_link = file_obj.get_file().file_path
  context.user_data['direct_download_file_name'] = file_name
  context.user_data['direct_download_link'] = file_link

  keyboard = [['Google Drive', 'Telegram']]
  reply_markup = ReplyKeyboardMarkup(keyboard, one_time_keyboard=True)
  update.message.reply_text('Where do you want to download the file?',
                            reply_markup=reply_markup)
  context.user_data['state'] = 'WAITING_FOR_DOWNLOAD_CHOICE'


# Main function to start the bot
def main():
  TOKEN = read_single_line_from_file(TELEGRAM_TOKEN_FILE)
  updater = Updater(TOKEN)
  dispatcher = updater.dispatcher

  dispatcher.add_handler(CommandHandler("start", start))
  dispatcher.add_handler(
      MessageHandler(Filters.regex(r'(http|https)://\S+'), upload_link))
  dispatcher.add_handler(
      MessageHandler(Filters.text & ~Filters.command, handle_download_choice))
  dispatcher.add_handler(MessageHandler(Filters.document, handle_file_upload))

  updater.start_polling()
  updater.idle()


if __name__ == '__main__':
  main()
